# Battle of Neighbourhood - Capstone Project

## Introduction

### Description and Discussion of Problem

Capitalising on its melting pot of cultures, Singapore is fast becoming one of Asia’s hit-list destinations.It attracts 17.4 million tourists everyyear. Apart from nice locations to see, its considered as an island for feasting and Shopping.

The "Fishtail" company is launching an travel club which will be targeting affluent visitors coming to Singapore. The key motivation for these shoppers are mainly in exploring the food, pubs, nightlife as well shopping for high end branded items. They have tied up with hotels in 3 areas of Singapore - Raffles, Orchard and Marina Bay Sands. They would want to set up a system where there members wanting to make booking can look at the profile of the vicinity and then decide where would they want to stay.       

## Objective

The aim of the study is two fold 


1) To  study the details  of  these 3 areas using Foursquare data and understand the profile of these areas. The aim of this project is to come up with the profiling  based on the most common places captured from Foursquare. These venue profile information will be used then to integarte into the "Fishtail" website. 

2) To understand the how hetergenous are the various areas and whether there are other areas that "Fishtail" can focus in future. 

## Data Set

To be able to do the above analysis following data sets has been used.

1) The main longitude and lattitude data for these areas has been acquired from wikipedia. 

2) Additionally, venue data has been acquired from foursquare API to understand the profile of different areas.

#### First we will start with reading the longitude and lattitude data and importing the relevant libraries that we need for further analysis and visualisation. 

In [2]:
# The code was removed by Watson Studio for sharing.

,Postal District,General Location,Area,Longitude,Latitude
0,1,"Raffles Place, Cecil, Marina, People’s Park",Raffles,1.2846,103.8510
1,2,"Anson, Tanjong Pagar",Anson,1.2780,103.8404
2,3,"Queenstown, Tiong Bahru",Queenstown,1.2942,103.7861
3,4,"Telok Blangah, Harbourfront",Harbour Front,1.2587,103.8187
4,5,"Pasir Panjang, Hong Leong Garden, Clementi New...",Pasir Panjang,1.3165,103.7574


In [3]:
df_data.columns

Index(['Postal District', 'General Location', 'Area', 'Longitude', 'Latitude'], dtype='object')

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  50.78 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.29 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.70 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  42.63 MB/s
Folium installed and imported!


In [14]:
# creating map of singapore using lattitude and longitude values 
from geopy.geocoders import Nominatim
import folium

address = 'Singapore'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Singapore using latitude and longitude values

map_sg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_data['Latitude'], df_data['Longitude'],df_data['Postal District'], df_data['Area']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sg)  
    
map_sg



#### Now, let's get the top 200 venues that are in all the areas within a radius of 500 meters around each of the locations

In [15]:
CLIENT_ID = 'FJ5ASPQT4OMXNO4OVJIE4HQAU0BIDQS2WCF2BEXW4R3LNIBX' # your Foursquare ID
CLIENT_SECRET = '0JE4DGK2Z2L5K2ANN42R2YROBD1ARGYCESU4PDIPZRSCQXKJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FJ5ASPQT4OMXNO4OVJIE4HQAU0BIDQS2WCF2BEXW4R3LNIBX
CLIENT_SECRET:0JE4DGK2Z2L5K2ANN42R2YROBD1ARGYCESU4PDIPZRSCQXKJ


In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## ORCHARD AREA 

In [19]:
LIMIT = 100
radius = 500

url_o = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.303, 
    103.8258, 
    radius, 
    LIMIT)
url_o # display URL

results_o = requests.get(url_o).json()
results_o

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_o = results_o['response']['groups'][0]['items']
    
nearby_venues_o = json_normalize(venues_o) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_o =nearby_venues_o.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_o['venue.categories'] = nearby_venues_o.apply(get_category_type, axis=1)

# clean columns
nearby_venues_o.columns = [col.split(".")[-1] for col in nearby_venues_o.columns]

nearby_venues_o.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_o.shape[0]))

nearby_venues_o["Area"]="Orchard"
nearby_venues_o.head()

23 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Basilico,Italian Restaurant,1.304074,103.824516,Orchard
1,Manhattan,Cocktail Bar,1.304911,103.825368,Orchard
2,The St. Regis Singapore,Hotel,1.305490,103.826074,Orchard
3,Dolcetto,Deli / Bodega,1.304726,103.825116,Orchard
4,Fat Cow,Japanese Restaurant,1.302958,103.824053,Orchard


## MARINA BAY SANDS

In [20]:
url_M = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.2834, 
    103.8607, 
    radius, 
    LIMIT)
url_M # display URL

results_M = requests.get(url_M).json()
results_M

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_M = results_M['response']['groups'][0]['items']
    
nearby_venues_M = json_normalize(venues_M) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_M =nearby_venues_M.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_M['venue.categories'] = nearby_venues_M.apply(get_category_type, axis=1)

# clean columns
nearby_venues_M.columns = [col.split(".")[-1] for col in nearby_venues_M.columns]

nearby_venues_M.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_M.shape[0]))

nearby_venues_M["Area"]="Marina Bay Sands"
nearby_venues_M.head()

    

60 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Marina Bay Sands Hotel,Hotel,1.283476,103.860313,Marina Bay Sands
1,Marina Bay Sands,Resort,1.283190,103.859124,Marina Bay Sands
2,Rooftop Infinity Pool,Hotel Pool,1.283868,103.860725,Marina Bay Sands
3,The Shoppes At Marina Bay Sands,Shopping Mall,1.283383,103.859157,Marina Bay Sands
4,Sands SkyPark,Roof Deck,1.284643,103.860887,Marina Bay Sands


In [21]:
# Little India

url_li = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3066, 
    103.8518, 
    radius, 
    LIMIT)
url_li # display URL

results_li = requests.get(url_li).json()
results_li

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_li = results_li['response']['groups'][0]['items']
    
nearby_venues_li = json_normalize(venues_li) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_li =nearby_venues_li.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_li['venue.categories'] = nearby_venues_li.apply(get_category_type, axis=1)

# clean columns
nearby_venues_li.columns = [col.split(".")[-1] for col in nearby_venues_li.columns]

nearby_venues_li.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_li.shape[0]))

nearby_venues_li["Area"]="Little India"
nearby_venues_li.head()

43 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Komala Vilas Restaurant,Vegetarian / Vegan Restaurant,1.306523,103.851848,Little India
1,Azmi Restaurant,Indian Restaurant,1.308256,103.853075,Little India
2,Mustard Restaurant,Indian Restaurant,1.308024,103.850508,Little India
3,Komala Vilas (Buffalo Rd),Indian Restaurant,1.306308,103.851158,Little India
4,Kailash Parbat,Indian Restaurant,1.308039,103.852660,Little India


In [22]:
# Anson
url_an = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.278, 
    103.8404, 
    radius, 
    LIMIT)
url_an # display URL

results_an = requests.get(url_an).json()
results_an

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_an = results_an['response']['groups'][0]['items']
    
nearby_venues_an = json_normalize(venues_an) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_an =nearby_venues_an.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_an['venue.categories'] = nearby_venues_an.apply(get_category_type, axis=1)

# clean columns
nearby_venues_an.columns = [col.split(".")[-1] for col in nearby_venues_an.columns]

nearby_venues_an.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_an.shape[0]))

nearby_venues_an["Area"]="Anson"
nearby_venues_an.head()

89 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Nylon Coffee Roasters,Coffee Shop,1.276657,103.840073,Anson
1,Binomio Spanish Restaurante,Spanish Restaurant,1.277713,103.842248,Anson
2,Man Man 鰻満 Japanese Unagi Restaurant (Man Man ...,Japanese Restaurant,1.278876,103.841514,Anson
3,D.Bespoke,Speakeasy,1.278680,103.840897,Anson
4,Neon Pigeon,Japanese Restaurant,1.278912,103.841543,Anson


In [23]:
# Queenstown

url_qu = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.2942, 
    103.7861, 
    radius, 
    LIMIT)
url_qu # display URL

results_qu = requests.get(url_qu).json()
results_qu

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_qu = results_qu['response']['groups'][0]['items']
    
nearby_venues_qu = json_normalize(venues_qu) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_qu =nearby_venues_qu.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_qu['venue.categories'] = nearby_venues_qu.apply(get_category_type, axis=1)

# clean columns
nearby_venues_qu.columns = [col.split(".")[-1] for col in nearby_venues_qu.columns]

nearby_venues_qu.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_qu.shape[0]))

nearby_venues_qu["Area"]="Queenstown"
nearby_venues_qu.head()

34 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,timbre+,Food Court,1.297759,103.787629,Queenstown
1,Iskina Cebu,Filipino Restaurant,1.297605,103.787517,Queenstown
2,NUH Medical Centre (One@KentRidge),Medical Center,1.293742,103.784358,Queenstown
3,Starbucks,Coffee Shop,1.294041,103.784483,Queenstown
4,Mr. Bean,Snack Place,1.294492,103.784332,Queenstown


In [25]:
url_hf = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.2587, 
    103.8187, 
    radius, 
    LIMIT)
url_hf # display URL

results_hf = requests.get(url_hf).json()
results_hf

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_hf = results_hf['response']['groups'][0]['items']
    
nearby_venues_hf = json_normalize(venues_hf) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_hf =nearby_venues_hf.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_hf['venue.categories'] = nearby_venues_hf.apply(get_category_type, axis=1)

# clean columns
nearby_venues_hf.columns = [col.split(".")[-1] for col in nearby_venues_hf.columns]

nearby_venues_hf.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_hf.shape[0]))

nearby_venues_hf["Area"]="Harbour Front"
nearby_venues_hf.head()

34 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,S.E.A. Aquarium,Aquarium,1.258445,103.820505,Harbour Front
1,Adventure Cove Waterpark,Water Park,1.258639,103.819451,Harbour Front
2,Hard Rock Cafe Sentosa,American Restaurant,1.257280,103.820067,Harbour Front
3,Resorts World Sentosa,Resort,1.256548,103.820338,Harbour Front
4,Starbucks Reserve Store,Coffee Shop,1.256458,103.820194,Harbour Front


In [26]:
url_pp = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3165, 
    103.7574, 
    radius, 
    LIMIT)
url_pp # display URL

results_pp = requests.get(url_pp).json()
results_pp

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_pp = results_pp['response']['groups'][0]['items']
    
nearby_venues_pp = json_normalize(venues_pp) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_pp =nearby_venues_pp.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_pp['venue.categories'] = nearby_venues_pp.apply(get_category_type, axis=1)

# clean columns
nearby_venues_pp.columns = [col.split(".")[-1] for col in nearby_venues_pp.columns]

nearby_venues_pp.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_pp.shape[0]))

nearby_venues_pp["Area"]="Pasir Panjang"
nearby_venues_pp.head()

8 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Ô boeuf à 6 pattes,French Restaurant,1.316416,103.759870,Pasir Panjang
1,Nobiro Japanese Restaurant & Sake bar,Japanese Restaurant,1.320411,103.756561,Pasir Panjang
2,Botannia,Residential Building (Apartment / Condo),1.318616,103.754168,Pasir Panjang
3,Bus Stop 20029 (Opp Hong Leong Gardens Shoppin...,Bus Line,1.317666,103.757565,Pasir Panjang
4,Tenis Court Carabelle,Tennis Court,1.317759,103.756651,Pasir Panjang


In [27]:
url_hs = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.272, 
    103.8417, 
    radius, 
    LIMIT)
url_hs # display URL

results_hs = requests.get(url_hs).json()
results_hs

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_hs = results_hs['response']['groups'][0]['items']
    
nearby_venues_hs = json_normalize(venues_hs) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_hs =nearby_venues_hs.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_hs['venue.categories'] = nearby_venues_hs.apply(get_category_type, axis=1)

# clean columns
nearby_venues_hs.columns = [col.split(".")[-1] for col in nearby_venues_hs.columns]

nearby_venues_hs.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_hs.shape[0]))

nearby_venues_hs["Area"]="High Street"
nearby_venues_hs.head()

23 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Amoy St Boon Kee Prawn Noodle,Food Court,1.272715,103.842434,High Street
1,Outram Park Ya Hua Rou Gu Cha | 欧南园亚华肉骨茶 (Outr...,Soup Place,1.272250,103.842241,High Street
2,DON DON DONKI,Discount Store,1.274742,103.843383,High Street
3,Two Men Bagel House,Bagel Shop,1.274147,103.844423,High Street
4,Jigger & Pony,Cocktail Bar,1.274394,103.843476,High Street


In [28]:
url_gm = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3027, 
    103.8651, 
    radius, 
    LIMIT)
url_gm # display URL

results_gm = requests.get(url_gm).json()
results_gm

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_gm = results_gm['response']['groups'][0]['items']
    
nearby_venues_gm = json_normalize(venues_gm) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_gm =nearby_venues_gm.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_gm['venue.categories'] = nearby_venues_gm.apply(get_category_type, axis=1)

# clean columns
nearby_venues_gm.columns = [col.split(".")[-1] for col in nearby_venues_gm.columns]

nearby_venues_gm.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_gm.shape[0]))

nearby_venues_gm["Area"]="Golden Mile"
nearby_venues_gm.head()

25 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,The Projector,Multiplex,1.302255,103.863553,Golden Mile
1,New Udon Thai Food (BBQ Steamboat),Thai Restaurant,1.303071,103.865223,Golden Mile
2,Beer Thai House Restaurant,Thai Restaurant,1.302445,103.864962,Golden Mile
3,Burgs By Project Warung,Burger Joint,1.303009,103.863833,Golden Mile
4,Golden Mile (Beach Road) Food Centre,Food Court,1.303145,103.864010,Golden Mile


In [29]:
url_bt = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3294, 
    103.8021, 
    radius, 
    LIMIT)
url_bt # display URL

results_bt = requests.get(url_bt).json()
results_bt

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_bt = results_bt['response']['groups'][0]['items']
    
nearby_venues_bt = json_normalize(venues_bt) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_bt =nearby_venues_bt.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_bt['venue.categories'] = nearby_venues_bt.apply(get_category_type, axis=1)

# clean columns
nearby_venues_bt.columns = [col.split(".")[-1] for col in nearby_venues_bt.columns]

nearby_venues_bt.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_bt.shape[0]))

nearby_venues_bt["Area"]="Bukit Timah"
nearby_venues_bt.head()

4 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Hwa Chong Swimming Complex,Pool,1.328480,103.802354,Bukit Timah
1,HCI Canteen,Food,1.325821,103.801922,Bukit Timah
2,MegaBites Namly Cafe,Snack Place,1.326200,103.799883,Bukit Timah
3,Hwa Chong Institution (College Section) Tennis...,Tennis Court,1.326218,103.805141,Bukit Timah


In [30]:
url_nv = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3209, 
    103.8424, 
    radius, 
    LIMIT)
url_nv # display URL

results_nv = requests.get(url_nv).json()
results_nv

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_nv = results_nv['response']['groups'][0]['items']
    
nearby_venues_nv = json_normalize(venues_nv) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_nv =nearby_venues_nv.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_nv['venue.categories'] = nearby_venues_nv.apply(get_category_type, axis=1)

# clean columns
nearby_venues_nv.columns = [col.split(".")[-1] for col in nearby_venues_nv.columns]

nearby_venues_nv.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_nv.shape[0]))

nearby_venues_nv["Area"]="Novena"
nearby_venues_nv.head()

56 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Ah Chew Desserts 阿秋甜品 (Ah Chew Desserts),Dessert Shop,1.318411,103.843714,Novena
1,Joe & Dough,Sandwich Place,1.320738,103.843819,Novena
2,La Ristrettos,Café,1.320699,103.844150,Novena
3,Courtyard by Marriott Singapore Novena,Hotel,1.320528,103.843221,Novena
4,Mama Patisserie,Bakery,1.321271,103.844129,Novena


In [31]:
url_sg = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3214, 
    103.8528, 
    radius, 
    LIMIT)
url_sg # display URL

results_sg = requests.get(url_sg).json()
results_sg

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_sg = results_sg['response']['groups'][0]['items']
    
nearby_venues_sg = json_normalize(venues_sg) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_sg =nearby_venues_sg.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_sg['venue.categories'] = nearby_venues_sg.apply(get_category_type, axis=1)

# clean columns
nearby_venues_sg.columns = [col.split(".")[-1] for col in nearby_venues_sg.columns]

nearby_venues_sg.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_sg.shape[0]))

nearby_venues_sg["Area"]="Serangoon"
nearby_venues_sg.head()

32 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,"""Combat"" Top Quality Durian",Dessert Shop,1.321567,103.852832,Serangoon
1,Sing Hon Loong Bakery,Bakery,1.322613,103.853204,Serangoon
2,Fragrance Rose Hotel Singapore,Hotel,1.321826,103.852735,Serangoon
3,新恒丰粿条摊 （夜市：鱼头炉）,Food,1.323469,103.853951,Serangoon
4,Whampoa Drive Market & Food Centre,Food Court,1.323087,103.854825,Serangoon


In [32]:
url_br = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3437, 
    103.8547, 
    radius, 
    LIMIT)
url_br # display URL

results_br = requests.get(url_br).json()
results_br

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_br = results_br['response']['groups'][0]['items']
    
nearby_venues_br = json_normalize(venues_br) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_br =nearby_venues_br.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_br['venue.categories'] = nearby_venues_br.apply(get_category_type, axis=1)

# clean columns
nearby_venues_br.columns = [col.split(".")[-1] for col in nearby_venues_br.columns]

nearby_venues_br.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_br.shape[0]))

nearby_venues_br["Area"]="Bradell"
nearby_venues_br.head()

21 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Kallang Park Connector,Trail,1.342255,103.856293,Bradell
1,Ong Yong Lee Eating House,Food Court,1.344985,103.856054,Bradell
2,Yalong Bay 亚龙湾地道小厨 (Yalong Bay),Chinese Restaurant,1.343159,103.851373,Bradell
3,Toa Payoh Lorong 8 Market & Food Centre,Market,1.340221,103.854221,Bradell
4,Al-Haji Noordhin Restaurant,Coffee Shop,1.345529,103.855055,Bradell


In [33]:
url_en = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.32, 
    103.8917, 
    radius, 
    LIMIT)
url_en # display URL

results_en = requests.get(url_en).json()
results_en

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_en = results_en['response']['groups'][0]['items']
    
nearby_venues_en = json_normalize(venues_en) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_en =nearby_venues_en.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_en['venue.categories'] = nearby_venues_en.apply(get_category_type, axis=1)

# clean columns
nearby_venues_en.columns = [col.split(".")[-1] for col in nearby_venues_en.columns]

nearby_venues_en.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_en.shape[0]))

nearby_venues_en["Area"]="Eunos"
nearby_venues_en.head()

46 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,GreenDot,Vegetarian / Vegan Restaurant,1.318651,103.892590,Eunos
1,Ramen Keisuke Tonkotsu King,Ramen Restaurant,1.318828,103.893122,Eunos
2,Jollibee,Fast Food Restaurant,1.318859,103.893144,Eunos
3,Gong Cha 贡茶 (Gong Cha),Bubble Tea Shop,1.318388,103.893458,Eunos
4,Stuff'd,Burrito Place,1.318662,103.892681,Eunos


In [34]:
url_kt = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.30, 
    103.9, 
    radius, 
    LIMIT)
url_kt # display URL

results_kt = requests.get(url_kt).json()
results_kt

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_kt = results_kt['response']['groups'][0]['items']
    
nearby_venues_kt = json_normalize(venues_kt) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_kt =nearby_venues_kt.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_kt['venue.categories'] = nearby_venues_kt.apply(get_category_type, axis=1)

# clean columns
nearby_venues_kt.columns = [col.split(".")[-1] for col in nearby_venues_kt.columns]

nearby_venues_kt.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_kt.shape[0]))

nearby_venues_kt["Area"]="Katong"
nearby_venues_kt.head()

22 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,3Bars @ Chinese Swimming Club,Bar,1.300655,103.900337,Katong
1,Dona Manis Cake Shop,Bakery,1.303446,103.901186,Katong
2,UpDog Studio,Yoga Studio,1.297414,103.900210,Katong
3,Delicious Boneless Chicken Rice,Chinese Restaurant,1.303555,103.900554,Katong
4,Ren Ren Desserts,Dessert Shop,1.303292,103.900927,Katong


In [35]:
url_bed = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3236, 
    103.9273, 
    radius, 
    LIMIT)
url_bed # display URL

results_bed = requests.get(url_bed).json()
results_bed

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_bed = results_bed['response']['groups'][0]['items']
    
nearby_venues_bed = json_normalize(venues_bed) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_bed =nearby_venues_bed.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_bed['venue.categories'] = nearby_venues_bed.apply(get_category_type, axis=1)

# clean columns
nearby_venues_bed.columns = [col.split(".")[-1] for col in nearby_venues_bed.columns]

nearby_venues_bed.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_bed.shape[0]))

nearby_venues_bed["Area"]="Bedok"
nearby_venues_bed.head()

60 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Ya Kun Kaya Toast 亞坤,Coffee Shop,1.324095,103.929198,Bedok
1,FairPrice Fínest,Supermarket,1.324140,103.929260,Bedok
2,Bedok Chwee Kueh 勿洛水粿,Chinese Restaurant,1.324903,103.930250,Bedok
3,Saffrons Cafeteria,Indian Restaurant,1.321524,103.924199,Bedok
4,Baker And Cook,Bakery,1.320861,103.924345,Bedok


In [36]:
url_ch = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.345, 
    103.9832, 
    radius, 
    LIMIT)
url_ch # display URL

results_ch = requests.get(url_ch).json()
results_ch

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_ch = results_ch['response']['groups'][0]['items']
    
nearby_venues_ch = json_normalize(venues_ch) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_ch =nearby_venues_ch.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_ch['venue.categories'] = nearby_venues_ch.apply(get_category_type, axis=1)

# clean columns
nearby_venues_ch.columns = [col.split(".")[-1] for col in nearby_venues_ch.columns]

nearby_venues_ch.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_ch.shape[0]))

nearby_venues_ch["Area"]="Changi"
nearby_venues_ch.head()

12 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Coffee Bean,Coffee Shop,1.341397,103.985704,Changi
1,Cabin Crew Taxi ( T2 ),Airport Terminal,1.344809,103.983371,Changi
2,SATS Inflight Catering Centre 1 Canteen,Café,1.344195,103.983178,Changi
3,Airport Boulevard,Road,1.345959,103.983836,Changi
4,Hugo Boss,Men's Store,1.347255,103.985423,Changi


In [37]:
url_tmp = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.372, 
    103.9473, 
    radius, 
    LIMIT)
url_tmp # display URL

results_tmp = requests.get(url_tmp).json()
results_tmp

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_tmp = results_tmp['response']['groups'][0]['items']
    
nearby_venues_tmp = json_normalize(venues_tmp) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_tmp =nearby_venues_tmp.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_tmp['venue.categories'] = nearby_venues_tmp.apply(get_category_type, axis=1)

# clean columns
nearby_venues_tmp.columns = [col.split(".")[-1] for col in nearby_venues_tmp.columns]

nearby_venues_tmp.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_tmp.shape[0]))

nearby_venues_tmp["Area"]="Tampines"
nearby_venues_tmp.head()

28 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Cottage Pies Cafe,Bakery,1.372933,103.949608,Tampines
1,Subway,Sandwich Place,1.372509,103.949420,Tampines
2,White Sands Shopping Centre,Shopping Mall,1.372433,103.949484,Tampines
3,Saizeriya Ristorante E Caffe,Italian Restaurant,1.372378,103.949261,Tampines
4,Cedele,Café,1.372687,103.949424,Tampines


In [38]:
url_png = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3638, 
    103.8662, 
    radius, 
    LIMIT)
url_png # display URL

results_png = requests.get(url_png).json()
results_png

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_png = results_png['response']['groups'][0]['items']
    
nearby_venues_png = json_normalize(venues_png) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_png =nearby_venues_png.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_png['venue.categories'] = nearby_venues_png.apply(get_category_type, axis=1)

# clean columns
nearby_venues_png.columns = [col.split(".")[-1] for col in nearby_venues_png.columns]

nearby_venues_png.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_png.shape[0]))

nearby_venues_png["Area"]="Punggol"
nearby_venues_png.head()

46 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Dessert Bowl 一碗甜品,Dessert Shop,1.364133,103.865056,Punggol
1,Chomp Chomp Food Centre,Food Court,1.364282,103.866342,Punggol
2,Chuan Heng Fish Ball Minced Meat Noodle,Noodle House,1.363396,103.866806,Punggol
3,Yogiyo Korean Fried Chicken,Korean Restaurant,1.364873,103.865387,Punggol
4,Bulldog Cafe Pub,Pub,1.364388,103.866024,Punggol


In [39]:
url_bis = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3691, 
    103.8454, 
    radius, 
    LIMIT)
url_bis # display URL

results_bis = requests.get(url_bis).json()
results_bis

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_bis = results_bis['response']['groups'][0]['items']
    
nearby_venues_bis = json_normalize(venues_bis) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_bis =nearby_venues_bis.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_bis['venue.categories'] = nearby_venues_bis.apply(get_category_type, axis=1)

# clean columns
nearby_venues_bis.columns = [col.split(".")[-1] for col in nearby_venues_bis.columns]

nearby_venues_bis.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_bis.shape[0]))

nearby_venues_bis["Area"]="Bishan"
nearby_venues_bis.head()

60 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Kam Jia Zhuang Restaurant,Asian Restaurant,1.368167,103.844118,Bishan
1,Subway,Sandwich Place,1.369136,103.847612,Bishan
2,MOS Burger,Burger Joint,1.369170,103.847831,Bishan
3,Old Chang Kee,Snack Place,1.369094,103.848389,Bishan
4,Bun Master,Bakery,1.369242,103.849031,Bishan


In [40]:
url_upt = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3206, 
    103.7762, 
    radius, 
    LIMIT)
url_upt # display URL

results_upt = requests.get(url_upt).json()
results_upt

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_upt = results_upt['response']['groups'][0]['items']
    
nearby_venues_upt = json_normalize(venues_upt) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_upt =nearby_venues_upt.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_upt['venue.categories'] = nearby_venues_upt.apply(get_category_type, axis=1)

# clean columns
nearby_venues_upt.columns = [col.split(".")[-1] for col in nearby_venues_upt.columns]

nearby_venues_upt.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_upt.shape[0]))

nearby_venues_upt["Area"]="Upper Bukit Timah"
nearby_venues_upt.head()

9 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,The French Ladle,French Restaurant,1.320320,103.779177,Upper Bukit Timah
1,Bus Stop 12041 (Aft Pandan Valley),Bus Station,1.320580,103.776576,Upper Bukit Timah
2,Basketball court @ pandan valley condo,Baseball Field,1.318930,103.777928,Upper Bukit Timah
3,Police KINS Training Camp gym,Gym / Fitness Center,1.320332,103.773563,Upper Bukit Timah
4,YOHA Canteen,Asian Restaurant,1.321733,103.778927,Upper Bukit Timah


In [41]:
url_jr = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3328, 
    103.7435, 
    radius, 
    LIMIT)
url_jr # display URL

results_jr = requests.get(url_jr).json()
results_jr

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_jr = results_jr['response']['groups'][0]['items']
    
nearby_venues_jr = json_normalize(venues_jr) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_jr =nearby_venues_jr.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_jr['venue.categories'] = nearby_venues_jr.apply(get_category_type, axis=1)

# clean columns
nearby_venues_jr.columns = [col.split(".")[-1] for col in nearby_venues_jr.columns]

nearby_venues_jr.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_jr.shape[0]))

nearby_venues_jr["Area"]="Jurong"
nearby_venues_jr.head()

67 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,UNIQLO,Clothing Store,1.333175,103.743160,Jurong
1,Tonkatsu by Ma Maison とんかつ マメゾン (Tonkatsu by M...,Japanese Restaurant,1.333668,103.742818,Jurong
2,MUJI 無印良品,Furniture / Home Store,1.333187,103.743064,Jurong
3,Johan Paris,Bakery,1.334083,103.742384,Jurong
4,Platinum Movie Suites,Movie Theater,1.333425,103.744113,Jurong


In [42]:
url_df = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3646, 
    103.7732, 
    radius, 
    LIMIT)
url_df # display URL

results_df = requests.get(url_df).json()
results_df

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_df = results_df['response']['groups'][0]['items']
    
nearby_venues_df = json_normalize(venues_df) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_df =nearby_venues_df.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_df['venue.categories'] = nearby_venues_df.apply(get_category_type, axis=1)

# clean columns
nearby_venues_df.columns = [col.split(".")[-1] for col in nearby_venues_df.columns]

nearby_venues_df.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_df.shape[0]))

nearby_venues_df["Area"]="Dairy Farm"
nearby_venues_df.head()

9 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,Dairy Farm Nature Park,Nature Preserve,1.364448,103.776364,Dairy Farm
1,Dairy Farm Pass,Bike Trail,1.362520,103.774753,Dairy Farm
2,Tenryu Japanese Dining,Japanese Restaurant,1.364970,103.769490,Dairy Farm
3,Car Park B | Dairy Farm Nature Park,Parking,1.364522,103.776376,Dairy Farm
4,Dairy Farm Quarry,Trail,1.361468,103.772575,Dairy Farm


In [43]:
url_kr = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.4252, 
    103.762, 
    radius, 
    LIMIT)
url_kr # display URL

results_kr = requests.get(url_kr).json()
results_kr
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_kr = results_kr['response']['groups'][0]['items']
    
nearby_venues_kr = json_normalize(venues_kr) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_kr =nearby_venues_kr.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_kr['venue.categories'] = nearby_venues_kr.apply(get_category_type, axis=1)

# clean columns
nearby_venues_kr.columns = [col.split(".")[-1] for col in nearby_venues_kr.columns]

nearby_venues_kr.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_kr.shape[0]))

nearby_venues_kr["Area"]="Kranji"
nearby_venues_kr.head()


12 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,KF1 Karting Circuit,Go Kart Track,1.423341,103.758710,Kranji
1,Singapore Turf Club,Racetrack,1.423217,103.762422,Kranji
2,Ananas Cafe,Café,1.425326,103.761853,Kranji
3,Bus Stop 45131 (Opp Kranji Stn),Bus Stop,1.425071,103.760958,Kranji
4,Causeway Link: Bus CW1,Bus Line,1.425541,103.761950,Kranji


In [44]:
url_ut = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.3277, 
    103.8407, 
    radius, 
    LIMIT)
url_ut # display URL

results_ut = requests.get(url_ut).json()
results_ut
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_ut = results_ut['response']['groups'][0]['items']
    
nearby_venues_ut = json_normalize(venues_ut) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_ut =nearby_venues_ut.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_ut['venue.categories'] = nearby_venues_ut.apply(get_category_type, axis=1)

# clean columns
nearby_venues_ut.columns = [col.split(".")[-1] for col in nearby_venues_ut.columns]

nearby_venues_ut.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_ut.shape[0]))

nearby_venues_ut["Area"]="Upper Thompson"
nearby_venues_ut.head()

15 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,龍發豆沙餅 Loong Fatt Tau Sar Piah,Bakery,1.326428,103.842327,Upper Thompson
1,Cafe de Hong Kong,Chinese Restaurant,1.326604,103.843380,Upper Thompson
2,Whampoa Keng Fish Head Steamboat Restaurant 黃埔...,Chinese Restaurant,1.326644,103.844945,Upper Thompson
3,Offside Futsal Park,Soccer Field,1.328586,103.840132,Upper Thompson
4,Sin Mee Eating House,Food Court,1.324871,103.841961,Upper Thompson


In [45]:
url_yis = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.4303, 
    103.8353, 
    radius, 
    LIMIT)
url_yis # display URL

results_yis = requests.get(url_yis).json()
results_yis
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_yis = results_yis['response']['groups'][0]['items']
    
nearby_venues_yis = json_normalize(venues_yis) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_yis =nearby_venues_yis.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_yis['venue.categories'] = nearby_venues_yis.apply(get_category_type, axis=1)

# clean columns
nearby_venues_yis.columns = [col.split(".")[-1] for col in nearby_venues_yis.columns]

nearby_venues_yis.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_yis.shape[0]))

nearby_venues_yis["Area"]="Yishun"
nearby_venues_yis.head()

41 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,KOI Cafè,Bubble Tea Shop,1.429787,103.836295,Yishun
1,Ding Tai Fung,Chinese Restaurant,1.429117,103.836091,Yishun
2,MOS Burger,Burger Joint,1.429413,103.836711,Yishun
3,928 Yishun Laksa,Noodle House,1.427399,103.837451,Yishun
4,Heng Hua Restaurant 兴化美食,Chinese Restaurant,1.427231,103.833268,Yishun


In [46]:
url_sel = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.4051, 
    103.8663, 
    radius, 
    LIMIT)
url_sel # display URL

results_sel = requests.get(url_sel).json()
results_sel
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_sel = results_sel['response']['groups'][0]['items']
    
nearby_venues_sel = json_normalize(venues_sel) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_sel =nearby_venues_sel.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_sel['venue.categories'] = nearby_venues_sel.apply(get_category_type, axis=1)

# clean columns
nearby_venues_sel.columns = [col.split(".")[-1] for col in nearby_venues_sel.columns]

nearby_venues_sel.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_sel.shape[0]))

nearby_venues_sel["Area"]="Seletar"
nearby_venues_sel.head()

6 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,The Summerhouse,English Restaurant,1.408038,103.868389,Seletar
1,YOUNGS Bar & Restaurant,Modern European Restaurant,1.407000,103.867589,Seletar
2,Wildseed Cafe,Café,1.408053,103.868398,Seletar
3,Bombardier Service Centre,Other Repair Shop,1.405130,103.864361,Seletar
4,Di Wei Teochew Restaurant,Chinese Restaurant,1.407455,103.868010,Seletar


In [47]:
url_rf = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    1.2846, 
    103.851, 
    radius, 
    LIMIT)
url_rf # display URL

results_rf = requests.get(url_rf).json()
results_rf
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues_rf = results_rf['response']['groups'][0]['items']
    
nearby_venues_rf = json_normalize(venues_rf) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_rf =nearby_venues_rf.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_rf['venue.categories'] = nearby_venues_rf.apply(get_category_type, axis=1)

# clean columns
nearby_venues_rf.columns = [col.split(".")[-1] for col in nearby_venues_rf.columns]

nearby_venues_rf.head(15)

print('{} venues were returned by Foursquare.'.format(nearby_venues_rf.shape[0]))

nearby_venues_rf["Area"]="Raffles"
nearby_venues_rf.head()

100 venues were returned by Foursquare.


,name,categories,lat,lng,Area
0,1-Altitude,Cocktail Bar,1.284794,103.851151,Raffles
1,CITY Hot Pot Shabu shabu,Hotpot Restaurant,1.284173,103.851585,Raffles
2,Virgin Active,Gym / Fitness Center,1.284608,103.850815,Raffles
3,CULINARYON,Comfort Food Restaurant,1.284876,103.850933,Raffles
4,The Salad Shop,Salad Place,1.285523,103.851177,Raffles


In [50]:
venues = pd.concat([nearby_venues_rf,nearby_venues_M,nearby_venues_o,nearby_venues_li,nearby_venues_qu,nearby_venues_an,nearby_venues_hf,nearby_venues_pp,nearby_venues_hs,nearby_venues_gm,nearby_venues_bt,
                           nearby_venues_nv,nearby_venues_sg,nearby_venues_br,nearby_venues_en,nearby_venues_kt,nearby_venues_bed,nearby_venues_ch,nearby_venues_tmp,nearby_venues_png,
                           nearby_venues_bis,nearby_venues_upt,nearby_venues_jr,nearby_venues_df,nearby_venues_kr,nearby_venues_ut, nearby_venues_yis,nearby_venues_sel])

In [51]:
#check the size of the resulting dataframe
print(venues.shape)
venues.head()

(985, 5)


,name,categories,lat,lng,Area
0,1-Altitude,Cocktail Bar,1.284794,103.851151,Raffles
1,CITY Hot Pot Shabu shabu,Hotpot Restaurant,1.284173,103.851585,Raffles
2,Virgin Active,Gym / Fitness Center,1.284608,103.850815,Raffles
3,CULINARYON,Comfort Food Restaurant,1.284876,103.850933,Raffles
4,The Salad Shop,Salad Place,1.285523,103.851177,Raffles


In [52]:
# one hot encoding
singapore_onehot = pd.get_dummies(venues[['categories']], prefix="", prefix_sep="")



In [53]:
# add neighborhood column back to dataframe
singapore_onehot['Area'] = venues['Area'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

#examine the new dataframe size after one hot encoding
print('{} rows were returned after one hot encoding.'.format(singapore_onehot.shape[0]))

#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
singapore_grouped = singapore_onehot.groupby('Area').mean().reset_index()

#examine the new dataframe size after one hot encoding
print('{} rows were returned after grouping.'.format(singapore_grouped.shape[0]))

985 rows were returned after one hot encoding.
28 rows were returned after grouping.


In [54]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 10

for hood in singapore_grouped['Area']:
    print("----"+hood+"----")
    temp = singapore_grouped[singapore_grouped['Area'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anson----
                 venue  freq
0                 Café  0.08
1  Japanese Restaurant  0.06
2          Coffee Shop  0.06
3    Korean Restaurant  0.04
4   Italian Restaurant  0.04
5         Cocktail Bar  0.04
6                Hotel  0.03
7     Ramen Restaurant  0.03
8     Asian Restaurant  0.03
9     Tapas Restaurant  0.03


----Bedok----
                  venue  freq
0                  Café  0.07
1           Supermarket  0.05
2           Coffee Shop  0.05
3            Food Court  0.05
4      Sushi Restaurant  0.05
5    Chinese Restaurant  0.03
6  Fast Food Restaurant  0.03
7   Fried Chicken Joint  0.03
8                Bakery  0.03
9     Indian Restaurant  0.03


----Bishan----
                  venue  freq
0           Coffee Shop  0.08
1            Food Court  0.07
2          Dessert Shop  0.07
3  Fast Food Restaurant  0.07
4       Bubble Tea Shop  0.05
5           Snack Place  0.03
6      Halal Restaurant  0.03
7        Sandwich Place  0.03
8   Japanese Restaurant  0.03
9   

In [55]:
#put into a pandas dataframe

#write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
areas_venues_sorted = pd.DataFrame(columns=columns)
areas_venues_sorted['Area'] = singapore_grouped['Area']

for ind in np.arange(singapore_grouped.shape[0]):
    areas_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

areas_venues_sorted.head(40)

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anson,Café,Japanese Restaurant,Coffee Shop,Cocktail Bar,Korean Restaurant,Italian Restaurant,Hotel,Asian Restaurant,Tapas Restaurant,French Restaurant
1,Bedok,Café,Coffee Shop,Food Court,Sushi Restaurant,Supermarket,Japanese Restaurant,Bakery,Fried Chicken Joint,Fast Food Restaurant,Indian Restaurant
2,Bishan,Coffee Shop,Dessert Shop,Food Court,Fast Food Restaurant,Bubble Tea Shop,Noodle House,Supermarket,Sushi Restaurant,Japanese Restaurant,Café
3,Bradell,Food Court,Seafood Restaurant,Coffee Shop,Dessert Shop,Bus Station,Noodle House,Chinese Restaurant,Trail,Cafeteria,Food Truck
4,Bukit Timah,Food,Tennis Court,Snack Place,Pool,Comfort Food Restaurant,Concert Hall,Filipino Restaurant,Field,Fast Food Restaurant,Exhibit
5,Changi,Bus Station,Airport Terminal,Road,Coffee Shop,Movie Theater,Tunnel,Men's Store,Airport Service,Airport,Café
6,Dairy Farm,Trail,Pool,Rest Area,Nature Preserve,Japanese Restaurant,Parking,Gym,Bike Trail,Electronics Store,Dumpling Restaurant
7,Eunos,Asian Restaurant,Fast Food Restaurant,Noodle House,Breakfast Spot,Supermarket,Café,Ramen Restaurant,Shopping Mall,Bakery,Food Court
8,Golden Mile,Thai Restaurant,BBQ Joint,Coffee Shop,Vietnamese Restaurant,Food Court,Italian Restaurant,Restaurant,Steakhouse,Burger Joint,Indonesian Restaurant
9,Harbour Front,Hotel,Performing Arts Venue,Coffee Shop,Theme Park,Seafood Restaurant,Italian Restaurant,Spa,Souvenir Shop,Snack Place,Shopping Mall


In [94]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 7

singapore_grouped_clustering = singapore_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 




array([2, 2, 2, 2, 1, 0, 2, 2, 2, 2], dtype=int32)

In [95]:
areas_venues_sorted.head(28)

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anson,Café,Japanese Restaurant,Coffee Shop,Cocktail Bar,Korean Restaurant,Italian Restaurant,Hotel,Asian Restaurant,Tapas Restaurant,French Restaurant
1,Bedok,Café,Coffee Shop,Food Court,Sushi Restaurant,Supermarket,Japanese Restaurant,Bakery,Fried Chicken Joint,Fast Food Restaurant,Indian Restaurant
2,Bishan,Coffee Shop,Dessert Shop,Food Court,Fast Food Restaurant,Bubble Tea Shop,Noodle House,Supermarket,Sushi Restaurant,Japanese Restaurant,Café
3,Bradell,Food Court,Seafood Restaurant,Coffee Shop,Dessert Shop,Bus Station,Noodle House,Chinese Restaurant,Trail,Cafeteria,Food Truck
4,Bukit Timah,Food,Tennis Court,Snack Place,Pool,Comfort Food Restaurant,Concert Hall,Filipino Restaurant,Field,Fast Food Restaurant,Exhibit
5,Changi,Bus Station,Airport Terminal,Road,Coffee Shop,Movie Theater,Tunnel,Men's Store,Airport Service,Airport,Café
6,Dairy Farm,Trail,Pool,Rest Area,Nature Preserve,Japanese Restaurant,Parking,Gym,Bike Trail,Electronics Store,Dumpling Restaurant
7,Eunos,Asian Restaurant,Fast Food Restaurant,Noodle House,Breakfast Spot,Supermarket,Café,Ramen Restaurant,Shopping Mall,Bakery,Food Court
8,Golden Mile,Thai Restaurant,BBQ Joint,Coffee Shop,Vietnamese Restaurant,Food Court,Italian Restaurant,Restaurant,Steakhouse,Burger Joint,Indonesian Restaurant
9,Harbour Front,Hotel,Performing Arts Venue,Coffee Shop,Theme Park,Seafood Restaurant,Italian Restaurant,Spa,Souvenir Shop,Snack Place,Shopping Mall


In [96]:
singapore_merged=df_data
singapore_merged['Cluster Labels'] = kmeans.labels_

singapore_merged=singapore_merged.join(areas_venues_sorted.set_index('Area'), on='Area')
singapore_merged.head()

,Postal District,General Location,Area,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Raffles Place, Cecil, Marina, People’s Park",Raffles,1.2846,103.8510,2,Café,Chinese Restaurant,Japanese Restaurant,Italian Restaurant,Cocktail Bar,Gym / Fitness Center,Hotel,Bar,Salad Place,Yoga Studio
1,2,"Anson, Tanjong Pagar",Anson,1.2780,103.8404,2,Café,Japanese Restaurant,Coffee Shop,Cocktail Bar,Korean Restaurant,Italian Restaurant,Hotel,Asian Restaurant,Tapas Restaurant,French Restaurant
2,3,"Queenstown, Tiong Bahru",Queenstown,1.2942,103.7861,2,Food Court,Coffee Shop,Sandwich Place,Noodle House,Café,Bus Station,Metro Station,Beer Bar,Food,Filipino Restaurant
3,4,"Telok Blangah, Harbourfront",Harbour Front,1.2587,103.8187,2,Hotel,Performing Arts Venue,Coffee Shop,Theme Park,Seafood Restaurant,Italian Restaurant,Spa,Souvenir Shop,Snack Place,Shopping Mall
4,5,"Pasir Panjang, Hong Leong Garden, Clementi New...",Pasir Panjang,1.3165,103.7574,1,BBQ Joint,Residential Building (Apartment / Condo),Bus Line,Japanese Restaurant,Italian Restaurant,French Restaurant,Gym,Tennis Court,Electronics Store,Diner


# Results

In [97]:
# Cluster 1 for Singapore
singapore_merged.loc[singapore_merged['Cluster Labels'] == 0, singapore_merged.columns[[2] + list(range(5, singapore_merged.shape[1]))]]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,High Street,0,Japanese Restaurant,Cocktail Bar,Coffee Shop,Hotel,Café,Soup Place,Pier,Food Court,Discount Store,Salad Place
20,Upper Bukit Timah,0,Bus Station,Gym,Convenience Store,Gym / Fitness Center,French Restaurant,Asian Restaurant,Baseball Field,Yoga Studio,Field,Fast Food Restaurant
24,Upper Thompson,0,Chinese Restaurant,Food Court,Bakery,Plaza,Bus Line,Soccer Field,Fast Food Restaurant,Exhibit,English Restaurant,Electronics Store
25,Yishun,0,Chinese Restaurant,Supermarket,Bubble Tea Shop,Fast Food Restaurant,Food Court,Coffee Shop,Hainan Restaurant,Mexican Restaurant,Sushi Restaurant,Video Game Store


In [98]:
# Cluster 2 for Singapore
singapore_merged.loc[singapore_merged['Cluster Labels'] == 1, singapore_merged.columns[[2] + list(range(5, singapore_merged.shape[1]))]]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Pasir Panjang,1,BBQ Joint,Residential Building (Apartment / Condo),Bus Line,Japanese Restaurant,Italian Restaurant,French Restaurant,Gym,Tennis Court,Electronics Store,Diner


In [99]:
# Cluster 3 for Singapore
singapore_merged.loc[singapore_merged['Cluster Labels'] == 2, singapore_merged.columns[[2] + list(range(5, singapore_merged.shape[1]))]]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Raffles,2,Café,Chinese Restaurant,Japanese Restaurant,Italian Restaurant,Cocktail Bar,Gym / Fitness Center,Hotel,Bar,Salad Place,Yoga Studio
1,Anson,2,Café,Japanese Restaurant,Coffee Shop,Cocktail Bar,Korean Restaurant,Italian Restaurant,Hotel,Asian Restaurant,Tapas Restaurant,French Restaurant
2,Queenstown,2,Food Court,Coffee Shop,Sandwich Place,Noodle House,Café,Bus Station,Metro Station,Beer Bar,Food,Filipino Restaurant
3,Harbour Front,2,Hotel,Performing Arts Venue,Coffee Shop,Theme Park,Seafood Restaurant,Italian Restaurant,Spa,Souvenir Shop,Snack Place,Shopping Mall
6,Golden Mile,2,Thai Restaurant,BBQ Joint,Coffee Shop,Vietnamese Restaurant,Food Court,Italian Restaurant,Restaurant,Steakhouse,Burger Joint,Indonesian Restaurant
7,Little India,2,Indian Restaurant,Vegetarian / Vegan Restaurant,Hostel,Chinese Restaurant,BBQ Joint,Arts & Crafts Store,Café,Restaurant,Cantonese Restaurant,Neighborhood
8,Orchard,2,Hotel,Chinese Restaurant,Japanese Restaurant,Café,Steakhouse,Restaurant,Supermarket,Korean Restaurant,Bar,Italian Restaurant
9,Bukit Timah,2,Food,Tennis Court,Snack Place,Pool,Comfort Food Restaurant,Concert Hall,Filipino Restaurant,Field,Fast Food Restaurant,Exhibit
10,Novena,2,Coffee Shop,Café,Chinese Restaurant,Italian Restaurant,Japanese Restaurant,Thai Restaurant,Sandwich Place,Bakery,Restaurant,Ramen Restaurant
11,Serangoon,2,Chinese Restaurant,Asian Restaurant,Food,Soup Place,Seafood Restaurant,Food Court,Dessert Shop,Indian Restaurant,Fast Food Restaurant,Hotel


In [100]:
# Cluster 4 for Singapore
singapore_merged.loc[singapore_merged['Cluster Labels'] == 3, singapore_merged.columns[[2] + list(range(5, singapore_merged.shape[1]))]]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Punggol,3,Chinese Restaurant,Dessert Shop,Noodle House,Bakery,Korean Restaurant,Thai Restaurant,Café,Italian Restaurant,Food Court,Coffee Shop


In [101]:
# Cluster 5 for Singapore
singapore_merged.loc[singapore_merged['Cluster Labels'] == 4, singapore_merged.columns[[2] + list(range(5, singapore_merged.shape[1]))]]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Seletar,4,Other Repair Shop,Food & Drink Shop,Chinese Restaurant,English Restaurant,Café,Modern European Restaurant,Diner,Field,Fast Food Restaurant,Exhibit


In [102]:
# Cluster 6 for Singapore
singapore_merged.loc[singapore_merged['Cluster Labels'] == 5, singapore_merged.columns[[2] + list(range(5, singapore_merged.shape[1]))]]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Dairy Farm,5,Trail,Pool,Rest Area,Nature Preserve,Japanese Restaurant,Parking,Gym,Bike Trail,Electronics Store,Dumpling Restaurant


In [103]:
# Cluster 7 for Singapore
singapore_merged.loc[singapore_merged['Cluster Labels'] == 6, singapore_merged.columns[[2] + list(range(5, singapore_merged.shape[1]))]]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Katong,6,Chinese Restaurant,Yoga Studio,Theme Park,Board Shop,Bike Rental / Bike Share,Bar,Bakery,Café,Restaurant,Thai Restaurant


## Observations and Conclusions

Based on cluster analysis by using Foursquare venue data we see the various locations are very homogenous in terms of presence of Eating & Dining (E&B) venues. Except few areas like Dairy Farm, Seletar seems to be less dominated by E&B. 

However, looking at the top venues in each area and profile we can see clear trend of domination of high end malls in the 2 areas - Raffles, Marina Bay Sands, whereas Marina Bay and Orchard more popular for yoga studio, pubs/bars and international cuisine restaurants. This information can be integarted in the "Fishtail" booking site to be able to guide their customers on choice of location for stay.  